In [1]:
import numpy as np
import threehalf_qe_cmc as threehalf
import time
from tqdm import tqdm
import pyfeng as pf
import matplotlib.pyplot as plt

In [2]:
# parameters
strike = [100.0, 140.0, 70.0]
forward = 100
delta = [1, 1/2, 1/4, 1/8, 1/16, 1/32]

case = np.zeros([3, 6])
case[0] = [1,   0.5, -0.9, 10, 0.04, np.sqrt(0.04)]
case[1] = [0.9, 0.3, -0.5, 15, 0.04, np.sqrt(0.04)]
case[2] = [1,   1,   -0.3, 5,  0.09, np.sqrt(0.09)]

In [3]:
# compute cmc price
for i in range(3):
    vov, kappa, rho, texp, theta, sigma = case[i]
    start = time.time()

    threehalf_cmc_qe = threehalf.ThreehalfQECondMC(vov=vov, kappa=kappa, rho=rho, theta=theta)
    price_cmc = np.zeros([len(delta), len(strike)])
    for d in range(len(delta)):
        price_cmc[d, :] = threehalf_cmc_qe.price(strike, forward, texp, sigma=sigma, delta=delta[d], path=1e5, seed=123456)

    end = time.time()
    
    print('Case %s:' % i)
    print(price_cmc)
    print('Running time is %.3f seconds.' % (end - start) + '\n')

Case 0:
[[22.95314785 10.44093783 38.98364955]
 [23.2425543  10.67162543 39.26731165]
 [23.20965635 10.64143576 39.21865023]
 [22.93527518 10.4758762  38.87971674]
 [22.9298084  10.47613694 38.88556212]
 [23.12806844 10.56484306 39.16893668]]
Running time is 35.341 seconds.

Case 1:
[[28.38562985 16.58649523 42.55740431]
 [28.42977766 16.61705767 42.61118032]
 [28.30786768 16.53990973 42.44717881]
 [28.30763704 16.53355657 42.45931412]
 [28.2822572  16.51219277 42.43468726]
 [28.37245592 16.57440991 42.54708381]]
Running time is 49.980 seconds.

Case 2:
[[25.20424465 13.66791193 39.93642084]
 [25.17249128 13.64813567 39.8960167 ]
 [25.20448092 13.66501972 39.94017581]
 [25.19447764 13.66509173 39.91560506]
 [25.12647967 13.62566786 39.82080577]
 [25.14555403 13.63602067 39.85066992]]
Running time is 16.496 seconds.



In [4]:
# compute std error of cmc price
n = 50
for i in range(3):
    start = time.time()
    vov, kappa, rho, texp, theta, sigma = case[i]

    threehalf_cmc_qe = threehalf.ThreehalfQECondMC(vov=vov, kappa=kappa, rho=rho, theta=theta)
    price_cmc = np.zeros([len(delta), len(strike), n])
    for j in tqdm(range(n)):
        for d in range(len(delta)):
            price_cmc[d, :, j] = threehalf_cmc_qe.price(strike, forward, texp, sigma=sigma, delta=delta[d], path=1e4)
            
    end = time.time()
    print('Case %s:' % i)
    print(price_cmc.mean(axis=2))
    print(price_cmc.std(axis=2))
    print('Running time is %.3f seconds.' % (end - start) + '\n')

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

Case 0:
[[23.1180362  10.57021836 39.13898791]
 [23.00578864 10.50122431 39.00027561]
 [23.05420493 10.53915821 39.04271169]
 [23.0806978  10.54469941 39.09342364]
 [23.01146571 10.51346128 38.99223514]
 [23.11379709 10.56798662 39.13715861]]
[[0.3610299  0.23883667 0.45158587]
 [0.26770007 0.1947411  0.32419022]
 [0.34735592 0.23349578 0.44609284]
 [0.3215627  0.22174824 0.4019566 ]
 [0.37695507 0.25665622 0.45714471]
 [0.28417037 0.17830129 0.36685526]]
Running time is 157.822 seconds.



 82%|███████████████████████████████████████████████████████████████████▏              | 41/50 [03:13<00:42,  4.75s/it]D:\Anaconda\lib\site-packages\pyfeng\bsm.py:54: RuntimeWarning: divide by zero encountered in log
  d1 = np.log(fwd / strike) / sigma_std
  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

Case 1:
[[28.36920023 16.57178558 42.54086293]
 [28.33348986 16.54828733 42.4946609 ]
 [28.315051   16.53790992 42.46771966]
 [28.38376151 16.58435552 42.5574009 ]
 [28.34221745 16.55247302 42.50977416]
 [28.35826204 16.56170042 42.53169188]]
[[0.22466811 0.14789248 0.29588545]
 [0.17594428 0.11507351 0.23286878]
 [0.21760176 0.14293305 0.28821106]
 [0.18762953 0.12326418 0.2477513 ]
 [0.18962096 0.12331284 0.25284861]
 [0.20052613 0.13145583 0.26263955]]
Running time is 235.675 seconds.



100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:20<00:00,  1.61s/it]

Case 2:
[[25.18709733 13.65551927 39.91595578]
 [25.18545002 13.65523695 39.91335104]
 [25.16089718 13.64610725 39.87054952]
 [25.19426419 13.66236572 39.92150427]
 [25.19504365 13.66231536 39.9239208 ]
 [25.18438898 13.65793022 39.90495725]]
[[0.08231093 0.04706323 0.11922582]
 [0.10169121 0.057242   0.14881877]
 [0.08833229 0.04877618 0.13017   ]
 [0.0855555  0.04710347 0.12661279]
 [0.10935767 0.06174692 0.1577052 ]
 [0.09864698 0.05431325 0.14581347]]
Running time is 80.377 seconds.

